# Librerias

In [8]:
import pandas as pd
import numpy as np
import requests
import json
import time
from dotenv import dotenv_values
import utils as u
import time
from datetime import datetime, timedelta

pd.set_option('max_colwidth', None)

### **GOOGLE MAPS:** Nearby Search

In [5]:
config = dotenv_values(".env") # Archivo de API Key oculto el archivo .env

In [ ]:
''' 
Este bucle, nos ayuda a extraer desde la API de Google Maps, los distintos IDs (restaurantes), que existe en la ciudad dentro de la M30.

El bucle consiste en:

Determinar los valores máximos y minimos de la latitud y longitud. Mirar los puntos extremos de cada medida.

El bucle va iterar por cada latitud y logitud ira aumentando en un step de 0.01. Cada uno de estas lat y lon realizará la consulta en la API
con un radio de 1000 metros y extraerá todos los id, incluyendo los de otras páginas (next_page_token) Google Maps solo tiene un max de 3 páginas.

Por último, hacemos un list comprehension en diccionarios para crear un diccionario que la clave es el place_id.

'''

lugares = []

norte = 40.485657
sur = 40.389104
este = -3.659151
oeste = -3.749617
step = 0.01


for lat in np.arange(sur, norte, step):
    for lon in np.arange(oeste, este, step):
        url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
        params = {'location': f'{lat},{lon}',
          'radius': 1000,
          'type': 'restaurant',
          'key':config['access_key']}

        for page in range(3):
            res = requests.get(url, params=params)
            #print(res)
            data = res.json()
            lugares += data.get('results', [])
            next_page_token = data.get('next_page_token')
            if next_page_token is not None:
                time.sleep(2)
                params = {'pagetoken': next_page_token, 'key': config['access_key']}
            else:
                break

lugares_unicos = {lugar['place_id']: lugar for lugar in lugares}

In [ ]:
with open('../data/raw/lugares_madrid_raw.csv', 'w') as file: # Guardamos el archivo en raw
     file.write(json.dumps(lugares_unicos))

In [ ]:
lugaresdf = u.json_to_dataframe(lugares_unicos) # Transformamos el json a un Data Frame

In [ ]:
lugaresdf.to_csv('../data/raw/lugares_madrid.csv', index=False) # Guardamos el archivo final

### **GOOGLE MAPS:** Places Detail

In [3]:
lugaresdf = pd.read_csv('../data/raw/lugares_madrid.csv')
lugaresdf.head(1)

,nombre,id,lat,lon
0,Arrocería Aynaelda,ChIJu0clrRCIQQ0R00GSTWpDKZQ,40.395986,-3.754164


In [9]:
''' 
Este bucle, nos ayuda a extraer desde la API de Google Maps de los IDs (restaurantes) antes sacados, el detalle de cada lugar.

El bucle consiste en:

Iterar por cada place_id, consultar en el api los detalles de los campos incluidos en la variable fields.
Y por último, gardarlos en una lista detalle_sitios.

'''

fields = ",".join(["place_id", "formatted_address",
    "wheelchair_accessible_entrance","opening_hours",
    "delivery", "dine_in", "editorial_summary",
    "price_level", "rating", "reservable", "reviews",
    "serves_beer", "serves_breakfast", "serves_brunch",
    "serves_dinner", "serves_lunch", "serves_vegetarian_food",
    "serves_wine", "takeout", "user_ratings_total","types"
])

detalle_sitios = []

for i in lugaresdf['id']:
    url = 'https://maps.googleapis.com/maps/api/place/details/json'
    
    params = {'place_id': i,
            'fields': fields,
            'language': 'en',
            'key':config['access_key']}

    try:
        details = requests.get(url, params=params, timeout=10)
        details.raise_for_status()
        resultado = details.json().get('result', {})
        detalle_sitios.append(resultado)
    except Exception as e:
        print(f"Error con ID {i}: {e}")
        detalle_sitios.append({})
    time.sleep(0.2)

Error con ID ChIJpYflyTYmQg0RVix8ZBBSzMI: HTTPSConnectionPool(host='maps.googleapis.com', port=443): Read timed out. (read timeout=10)


In [10]:
with open('../data/raw/detalle_sitios_raw.csv', 'w') as file: # Guardamos el archivo en raw
     file.write(json.dumps(detalle_sitios))

In [11]:
''' 
Guardamos los reviews en otro dataframe para procesar con NLP.
'''
rev = {'place_id':[],
       'rating_rev':[],
       'text':[]}

for i in detalle_sitios:
    try:
        reviews = i['reviews']
        
        for j in reviews:
            rev['place_id'].append(i.get('place_id')) 
            rev['rating_rev'].append(j.get('rating', ''))
            rev['text'].append(j.get('text', ''))
    except:
        continue

df_reviews = pd.DataFrame(rev) # Guardamos el resultado en un DataFrame

In [12]:
df_reviews.to_csv('../data/raw/detalle_reviews.csv', index=False) # Guardamos el archivo final

In [17]:
''' 
Este bucle, nos ayuda a extraer desde el json del detalle de sitios los distintos atributos.

'''

df_detalle = {  'place_id':[],
                'dine_in':[],
                'summary':[],
                'address':[],
                'price_level':[],
                'rating':[],
                'reservable':[],
                'serves_beer':[],
                'serves_breakfast':[],
                'serves_brunch':[],
                'serves_dinner':[],
                'serves_lunch':[],
                'serves_vegetarian_food':[],
                'serves_wine':[],
                'takeout':[],
                'user_ratings_total':[],
                'delivery':[],
                'weelchair':[],
                'hours_open':[],
                'num_days_open':[],
                'open_weekends':[],
                'types':[]                          
                }

for i in detalle_sitios:
    df_detalle['place_id'].append(i.get('place_id'))
    df_detalle['dine_in'].append(i.get('dine_in', np.nan))
    df_detalle['address'].append(i.get('formatted_address', ''))
    df_detalle['rating'].append(i.get('rating', np.nan))
    df_detalle['user_ratings_total'].append(i.get('user_ratings_total', np.nan))
    df_detalle['delivery'].append(i.get('delivery', np.nan))
    df_detalle['summary'].append(i.get('editorial_summary', {}).get('overview', ''))
    df_detalle['price_level'].append(i.get('price_level', np.nan))
    df_detalle['reservable'].append(i.get('reservable', np.nan))
    df_detalle['serves_beer'].append(i.get('serves_beer', np.nan))
    df_detalle['serves_breakfast'].append(i.get('serves_breakfast', np.nan))
    df_detalle['serves_brunch'].append(i.get('serves_brunch', np.nan))
    df_detalle['serves_dinner'].append(i.get('serves_dinner', np.nan))
    df_detalle['serves_lunch'].append(i.get('serves_lunch', np.nan))
    df_detalle['serves_vegetarian_food'].append(i.get('serves_vegetarian_food', np.nan))
    df_detalle['serves_wine'].append(i.get('serves_wine', np.nan))
    df_detalle['takeout'].append(i.get('takeout', np.nan))
    df_detalle['weelchair'].append(i.get('wheelchair_accessible_entrance', np.nan))
    df_detalle['types'].append(i.get('types', ''))


    total_hours = 0
    open_weekend = 0
    num_day_open = set()
    try:
        periods = i['opening_hours']['periods']
        for period in periods:
            open_day = period['open']['day']
            open_time = period['open']['time']
            close_time = period['close']['time']

            open_hour = int(open_time[:2])
            open_minute = int(open_time[2:])

            close_hour = int(close_time[:2])
            close_minute = int(close_time[2:])

            open_dt = datetime.strptime(f"{open_hour}:{open_minute}", "%H:%M")
            close_dt = datetime.strptime(f"{close_hour}:{close_minute}", "%H:%M")

            if close_dt <= open_dt:
                close_dt += timedelta(days=1)

            hours_open = (close_dt - open_dt).total_seconds() / 3600
            total_hours += hours_open
            num_day_open.add(open_day)

            if open_day == 0 or open_day == 6:
                open_weekend += 1
            
    except:
        total_hours = np.nan

    num_day_open = len(num_day_open)
    df_detalle['num_days_open'].append(num_day_open)
    df_detalle['hours_open'].append(total_hours)
    if open_weekend>=2:
        df_detalle['open_weekends'].append(True) 
    else:
        df_detalle['open_weekends'].append(False) 


In [18]:
detalle_sitios_df = pd.DataFrame(df_detalle) #Guardamos el resultado en un DataFrame

In [19]:
detalle_sitios_df.to_csv('../data/raw/detalle_sitios.csv', index=False) # Guardamos el archivo final

### **Open StreetMap**

In [ ]:
''' 
Consulta de Open StreetMap de los restaurantes de España.

'''

overpass_url = "http://overpass-api.de/api/interpreter"


overpass_query = """ 
[out:json][timeout:60];

area["name"="España"]["admin_level"="2"]->.spain;

(
  node["amenity"="restaurant"](area.spain);
  way["amenity"="restaurant"](area.spain);
  relation["amenity"="restaurant"](area.spain);
);

out center;

"""


response = requests.get(overpass_url, params={'data': overpass_query})
print(response)
data = response.json()

<Response [200]>


In [3]:
''' 
Este bucle, nos ayuda a extraer desde el json del detalle de sitios los distintos atributos.

'''

street = {'nombre':[],
          'postal':[],
          'calle':[],
          'numero':[],
          'cocina':[],
          'lat':[],
          'lon':[],
          'wheelchair':[]}

for i in data['elements']:
    street['nombre'].append(i['tags'].get('name', ''))
    street['postal'].append(i['tags'].get('addr:postcode', ''))
    street['calle'].append(i['tags'].get('addr:street', ''))
    street['numero'].append(i['tags'].get('addr:housenumber', ''))
    street['cocina'].append(i['tags'].get('cuisine', ''))
    street['lat'].append(i.get('lat', ''))
    street['lon'].append(i.get('lon', ''))
    street['wheelchair'].append(i['tags'].get('wheelchair', ''))

street_df = pd.DataFrame(street)

In [5]:
street_df.to_csv('../data/raw/sitios_streetmap.csv', index=False) # Guardamos el archivo final

### **Ayuntamiento Madrid**

In [ ]:
''' 
Datos del Ayuntamiento de Madrid con distintos KPI por barrio, filtramos los datos del 2024.
'''

df_barrios = pd.read_csv('../data/raw/panel_indicadores_distritos_barrios.csv', sep=';', index_col='Orden')

In [277]:
df_barrios['indicador_completo'] = df_barrios['indicador_completo'].str.strip() # Quitamos los espacios en blanco de los indicadores

In [278]:
df_barrios = df_barrios[(df_barrios['cod_distrito'].notna())&(df_barrios['cod_barrio'].notna())] # quitamos los kpi que no estan por barrio

In [ ]:
''' 
Muchos de estos indicadores, viene repetidos por año y barrio por lo que creamos este proceso que primero:
- Calculamos para cada barrio e indicador el periodo panel max.
- Filtramos el dataset para quedarnos solo con los registros de su fecha max con el merge.
- Calculamos de este nuevo dataset filtrado para cada barrio e indicador el año maximo.
- Filtramos el dataset para quedarnos solo con los registros de su año max con el merge.

De esta forma logramos tenemos para cada barrio e indicador la informacion las reciente.
'''


val = df_barrios.groupby(['cod_barrio', 'indicador_completo'])[['Periodo panel']].max().reset_index()

df_barrios = pd.merge(df_barrios, val, how='inner', 
                      left_on=['cod_barrio', 'indicador_completo', 'Periodo panel'],
                        right_on=['cod_barrio', 'indicador_completo', 'Periodo panel'])

val2 = df_barrios.groupby(['cod_barrio', 'indicador_completo'])[['año']].max().reset_index()

df_barrios = pd.merge(df_barrios, val2, how='inner', 
                      left_on=['cod_barrio', 'indicador_completo', 'año'],
                        right_on=['cod_barrio', 'indicador_completo', 'año'])

In [280]:
''' 
Nos quedamos con las columnas que nos interesan.
'''

df_barrios.drop(['Periodo panel', 'ciudad',
                 'fecha_indicador', 'fuente_indicador', 
                 'categoría_1', 'categoría_2', 'indicador_nivel1',
                 'indicador_nivel2', 'indicador_nivel3'], inplace=True, axis=1)

In [281]:
df_barrios.to_csv('../data/raw/kpi_barrios_madrid.csv', index=False) # Guardamos el archivo final